<a href="https://colab.research.google.com/github/pascalghanimi/Injury-Prediction-in-Runners/blob/main/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Datenextraktion für XGBoost
import pickle

with open("features.pkl", "rb") as f:
  features = pickle.load(f)

with open("features_days.pkl", "rb") as f:
  features_days = pickle.load(f)

with open("features_weeks.pkl", "rb") as f:
  features_weeks = pickle.load(f)

with open("features_objective.pkl", "rb") as f:
  features_objective = pickle.load(f)

with open("features_subjective.pkl", "rb") as f:
  features_subjective = pickle.load(f)

with open("labels.pkl", "rb") as f:
  labels = pickle.load(f)

print(features[0][0]) # greift auf ersten Athleten und erste Reihe zu
print(features_days[0][0])
print(features_weeks[0])
print(labels[0])

print(len(features[0][0])) # Anzahl der Features

[ 0.77523982  0.48379555 -0.34174298  1.12482238  4.65573275 -0.30257114
 -0.36195376  1.13221076  0.88408664  0.98946749 -1.07576228 -0.58780279
 -0.34515758 -0.21498604 -0.25818757 -0.30257114 -0.36653747 -1.10832632
 -1.12373205 -1.11938273  0.76147321 -0.58880824 -0.33963599 -0.22015974
 -0.26519341  3.1128211  -0.36131219  0.94306135  0.88408664  0.88198911
 -1.08073515 -0.58890474 -0.33382237 -0.21223522 -0.26471133 -0.29617444
 -0.36131219 -1.10795815 -1.12373205 -1.11942205  0.7667035  -0.5913292
 -0.33382237 -0.22367068 -0.26509659 -0.29617444  2.42402301  0.57403001
  0.88989186  0.9965464   0.78053094  2.39956507  5.70866175 -0.21223522
 -0.26519341  3.3763886  -0.35067995  1.13636789  0.88989186  0.88771258
  0.78584128 -0.58219763 -0.3393889  -0.21762941 -0.25818757 -0.28968273
  2.19131074  0.96442407  0.89572629  0.67074481  0.37283858 -0.47040001
 -0.01179093  0.73505066  1.36358038  0.5749299   0.54093582  1.26319022
  2.68987335 -0.05530893  0.11981333  1.16084473  1.

In [2]:
import numpy as np

X = np.vstack([features[athlete_id] for athlete_id in features])
X_days = np.vstack([features_days[athlete_id] for athlete_id in features_days])
X_weeks = np.vstack([features_weeks[athlete_id] for athlete_id in features_weeks])
X_objective = np.vstack([features_objective[athlete_id] for athlete_id in features_objective])
X_subjective = np.vstack([features_subjective[athlete_id] for athlete_id in features_subjective])

Y = np.hstack([labels[athlete_id] for athlete_id in labels])


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.3, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)

X_train_days, X_temp_days, Y_train_days, Y_temp_days = train_test_split(X_days, Y, test_size=0.3, random_state=42)
X_val_days, X_test_days, Y_val_days, Y_test_days = train_test_split(X_temp_days, Y_temp_days, test_size=0.5, random_state=42)

X_train_weeks, X_temp_weeks, Y_train_weeks, Y_temp_weeks = train_test_split(X_weeks, Y, test_size=0.3, random_state=42)
X_val_weeks, X_test_weeks, Y_val_weeks, Y_test_weeks = train_test_split(X_temp_weeks, Y_temp_weeks, test_size=0.5, random_state=42)

X_train_subjective, X_temp_subjective, Y_train_subjective, Y_temp_subjective = train_test_split(X_subjective, Y, test_size=0.3, random_state=42)
X_val_subjective, X_test_subjective, Y_val_subjective, Y_test_subjective = train_test_split(X_temp_subjective, Y_temp_subjective, test_size=0.5, random_state=42)

X_train_objective, X_temp_objective, Y_train_objective, Y_temp_objective = train_test_split(X_objective, Y, test_size=0.3, random_state=42)
X_val_objective, X_test_objective, Y_val_objective, Y_test_objective = train_test_split(X_temp_objective, Y_temp_objective, test_size=0.5, random_state=42)

print(X_train.shape, X_val.shape, X_test.shape)
print(X_train_days.shape, X_val_days.shape, X_test_days.shape)
print(X_train_weeks.shape, X_val_weeks.shape, X_test_weeks.shape)
print(X_train_subjective.shape, X_val_subjective.shape, X_test_subjective.shape)
print(X_train_objective.shape, X_val_objective.shape, X_test_objective.shape)
print(Y_train.shape, Y_val.shape, Y_test.shape)


(29876, 139) (6402, 139) (6402, 139)
(29876, 70) (6402, 70) (6402, 70)
(29876, 69) (6402, 69) (6402, 69)
(29876, 48) (6402, 48) (6402, 48)
(29876, 91) (6402, 91) (6402, 91)
(29876,) (6402,) (6402,)


In [4]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 5.1 MB/s eta 0:00:00


In [5]:
def get_training_batch(X_train, Y_train, batch_size):
    injured_indices = np.where(Y_train == 1)[0]
    uninjured_indices = np.where(Y_train == 0)[0]

    injured_sample = np.random.choice(injured_indices, size=batch_size // 2, replace=True)
    uninjured_sample = np.random.choice(uninjured_indices, size=batch_size // 2, replace=True)

    selected_indices = np.concatenate([injured_sample, uninjured_sample])
    np.random.shuffle(selected_indices)

    X_batch = X_train[selected_indices]
    Y_batch = Y_train[selected_indices]

    return X_batch, Y_batch

In [6]:
import optuna
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import warnings

# Warnings ausblenden (optional, falls dich andere nicht stören)
warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn")


def objective(trial):
    params = {
        "objective": "binary:logistic",
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),  # ✅ Kein loguniform mehr
        "max_depth": trial.suggest_int("max_depth", 2, 10),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),  # ✅ Kein suggest_uniform mehr
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.01, 10, log=True),  # ✅ Kein loguniform mehr
        "reg_alpha": trial.suggest_float("reg_alpha", 0.01, 10, log=True)  # ✅ Kein loguniform mehr
    }

    X_batch, Y_batch = get_training_batch(X_train, Y_train, 2048)

    # Modell trainieren
    model = xgb.XGBClassifier(**params)
    model.fit(X_batch, Y_batch, eval_set=[(X_val, Y_val)], verbose=False)

    # AUC messen
    Y_pred = model.predict_proba(X_val)[:, 1]
    return roc_auc_score(Y_val, Y_pred)

# 🔥 Optuna-Optimierung starten
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)  # 50 Versuche für Optimierung

# Beste Hyperparameter ausgeben
print("Beste Hyperparameter:", study.best_params)


[I 2025-03-03 11:22:41,298] A new study created in memory with name: no-name-e780572b-93b1-46ec-8ee2-b5e1dc8416f1
[I 2025-03-03 11:23:31,050] Trial 0 finished with value: 0.7290542401284372 and parameters: {'learning_rate': 0.009291855781281052, 'max_depth': 10, 'subsample': 0.7171060944154687, 'colsample_bytree': 0.9960668873160612, 'n_estimators': 331, 'reg_lambda': 0.5854705051833753, 'reg_alpha': 1.377237424934184}. Best is trial 0 with value: 0.7290542401284372.
[I 2025-03-03 11:23:48,049] Trial 1 finished with value: 0.6950250097167779 and parameters: {'learning_rate': 0.05117534576350498, 'max_depth': 10, 'subsample': 0.8446403325890988, 'colsample_bytree': 0.9863734621167967, 'n_estimators': 298, 'reg_lambda': 0.010938056672951791, 'reg_alpha': 0.26609114858651844}. Best is trial 0 with value: 0.7290542401284372.
[I 2025-03-03 11:24:01,137] Trial 2 finished with value: 0.7191616813248412 and parameters: {'learning_rate': 0.012620307651969039, 'max_depth': 7, 'subsample': 0.8364

Beste Hyperparameter: {'learning_rate': 0.009291855781281052, 'max_depth': 10, 'subsample': 0.7171060944154687, 'colsample_bytree': 0.9960668873160612, 'n_estimators': 331, 'reg_lambda': 0.5854705051833753, 'reg_alpha': 1.377237424934184}


In [7]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.calibration import CalibratedClassifierCV

def train (X_train, Y_train, X_val, Y_val, X_test, Y_test):

  best_params = study.best_params
  params = best_params

  num_models = 9
  models = []
  batch_size = 2048

  for model_idx in range(num_models):
    X_batch, Y_batch = get_training_batch(X_train, Y_train, batch_size)

    model = xgb.XGBClassifier(**params)
    model.fit(X_batch, Y_batch, eval_set=[(X_val, Y_val)], verbose=False)
    Y_preds_test = model.predict_proba(X_test)[:, 1]
    test_auc = roc_auc_score(Y_test, Y_preds_test)

    print(f"\nModell {model_idx + 1} Ergebnisse:")
    print(f"Test-AUC: {test_auc:.4f}")
    models.append(model)

  calibrated_models = [
      CalibratedClassifierCV(m, method='sigmoid', cv="prefit").fit(X_val, Y_val) for m in models
  ]


  # Ensemble-AUC auf Wahrscheinlichkeiten
  def ensemble_predict(models, X):
    test_probas = np.stack([m.predict_proba(X)[:, 1] for m in models])
    return np.mean(test_probas, axis=0)



  Y_proba_test = ensemble_predict(calibrated_models, X_test)
  test_auc = roc_auc_score(Y_test, Y_proba_test)

  print(f"Test-AUC: {test_auc:.4f}")

  return calibrated_models, test_auc

In [8]:
# Training mit allen Featuren
xgb_models, _ = train(X_train, Y_train, X_val, Y_val, X_test, Y_test)


Modell 1 Ergebnisse:
Test-AUC: 0.6988

Modell 2 Ergebnisse:
Test-AUC: 0.7053

Modell 3 Ergebnisse:
Test-AUC: 0.7030

Modell 4 Ergebnisse:
Test-AUC: 0.7075

Modell 5 Ergebnisse:
Test-AUC: 0.7064

Modell 6 Ergebnisse:
Test-AUC: 0.7076

Modell 7 Ergebnisse:
Test-AUC: 0.6996

Modell 8 Ergebnisse:
Test-AUC: 0.6976

Modell 9 Ergebnisse:
Test-AUC: 0.6885


/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.1

Test-AUC: 0.7159


In [ ]:
# Training mit den Tagesdaten
xgb_models_days, _ = train(X_train_days, Y_train_days, X_val_days, Y_val_days, X_test_days, Y_test_days)


Modell 1 Ergebnisse:
Test-AUC: 0.6996

Modell 2 Ergebnisse:
Test-AUC: 0.6886

Modell 3 Ergebnisse:
Test-AUC: 0.6997

Modell 4 Ergebnisse:
Test-AUC: 0.6771

Modell 5 Ergebnisse:
Test-AUC: 0.7034

Modell 6 Ergebnisse:
Test-AUC: 0.7042

Modell 7 Ergebnisse:
Test-AUC: 0.7076

Modell 8 Ergebnisse:
Test-AUC: 0.7098

Modell 9 Ergebnisse:
Test-AUC: 0.6816


/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.1

Test-AUC des Ensembles: 0.7148


In [ ]:
# Training mit den Wochendaten
xgb_models_weeks, _ = train(X_train_weeks, Y_train_weeks, X_val_weeks, Y_val_weeks, X_test_weeks, Y_test_weeks)


Modell 1 Ergebnisse:
Test-AUC: 0.6731

Modell 2 Ergebnisse:
Test-AUC: 0.6876

Modell 3 Ergebnisse:
Test-AUC: 0.6762

Modell 4 Ergebnisse:
Test-AUC: 0.7017

Modell 5 Ergebnisse:
Test-AUC: 0.6612

Modell 6 Ergebnisse:
Test-AUC: 0.6668

Modell 7 Ergebnisse:
Test-AUC: 0.7078

Modell 8 Ergebnisse:
Test-AUC: 0.6836

Modell 9 Ergebnisse:
Test-AUC: 0.6911


/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.1

Test-AUC des Ensembles: 0.6995


In [ ]:
# Training mit den objektiven Daten
xgb_models_objective, _ = train(X_train_objective, Y_train_objective, X_val_objective, Y_val_objective, X_test_objective, Y_test_objective)


Modell 1 Ergebnisse:
Test-AUC: 0.6893

Modell 2 Ergebnisse:
Test-AUC: 0.7130

Modell 3 Ergebnisse:
Test-AUC: 0.6800

Modell 4 Ergebnisse:
Test-AUC: 0.7061

Modell 5 Ergebnisse:
Test-AUC: 0.7177

Modell 6 Ergebnisse:
Test-AUC: 0.6869

Modell 7 Ergebnisse:
Test-AUC: 0.7009

Modell 8 Ergebnisse:
Test-AUC: 0.7077

Modell 9 Ergebnisse:
Test-AUC: 0.6983


/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.1

Test-AUC des Ensembles: 0.7189


In [ ]:
# Training mit den subjektiven Daten
xgb_models_subjective, _ = train(X_train_subjective, Y_train_subjective, X_val_subjective, Y_val_subjective, X_test_subjective, Y_test_subjective)


Modell 1 Ergebnisse:
Test-AUC: 0.6598

Modell 2 Ergebnisse:
Test-AUC: 0.6324

Modell 3 Ergebnisse:
Test-AUC: 0.6615

Modell 4 Ergebnisse:
Test-AUC: 0.6523

Modell 5 Ergebnisse:
Test-AUC: 0.6760

Modell 6 Ergebnisse:
Test-AUC: 0.6493

Modell 7 Ergebnisse:
Test-AUC: 0.6775

Modell 8 Ergebnisse:
Test-AUC: 0.6500

Modell 9 Ergebnisse:
Test-AUC: 0.6725


/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.1

Test-AUC des Ensembles: 0.6823


In [ ]:
amount_of_trainings = 2
all_auc_scores = []
auc_scores_days = []
auc_scores_weeks = []
auc_scores_objective = []
auc_scores_subjective = []

def calculate_average_auc_over_10_rounds(train_fn, auc_scores_array):
  for training in range(amount_of_trainings):
    _, auc = train_fn()
    auc_scores_array.append(auc)
  mean_auc_score = np.mean(auc_scores_array)
  std_auc_score = np.std(auc_scores_array)
  print(f"Mean AUC Score: {mean_auc_score}")
  print(f"Standard Deviation of AUC Scores: {std_auc_score}")

calculate_average_auc_over_10_rounds(lambda: train(X_train, Y_train, X_val, Y_val, X_test, Y_test), all_auc_scores)
calculate_average_auc_over_10_rounds(lambda: train(X_train_days, Y_train_days, X_val_days, Y_val_days, X_test_days, Y_test_days), auc_scores_days)
calculate_average_auc_over_10_rounds(lambda: train(X_train_weeks, Y_train_weeks, X_val_weeks, Y_val_weeks, X_test_weeks, Y_test_weeks), auc_scores_weeks)
calculate_average_auc_over_10_rounds(lambda: train(X_train_objective, Y_train_objective, X_val_objective, Y_val_objective, X_test_objective, Y_test_objective), auc_scores_objective)
calculate_average_auc_over_10_rounds(lambda: train(X_train_subjective, Y_train_subjective, X_val_subjective, Y_val_subjective, X_test_subjective, Y_test_subjective), auc_scores_subjective)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Stil für moderne Diagramme setzen
plt.style.use("seaborn-v0_8-darkgrid")

# Boxplot für alle Feature-Sets
plt.figure(figsize=(10, 6))
box = plt.boxplot(
    [all_auc_scores, auc_scores_days, auc_scores_weeks, auc_scores_objective, auc_scores_subjective],
    labels=["All Features", "Days", "Weeks", "Objective", "Subjective"],
    patch_artist=True,
    boxprops=dict(facecolor="lightblue", color="black"),
    medianprops=dict(color="darkred", linewidth=2),
    whiskerprops=dict(color="black", linewidth=1.5),
    capprops=dict(color="black", linewidth=1.5)
)
plt.ylabel("AUC Score", fontsize=12, fontweight="bold")
plt.xticks(fontsize=11)
plt.yticks(fontsize=11)
plt.title("Vergleich der AUC Scores über verschiedene Feature-Sets", fontsize=14, fontweight="bold")
plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.show()

# Mittelwerte berechnen
mean_auc_scores = {
    "All Features": np.mean(all_auc_scores),
    "Days": np.mean(auc_scores_days),
    "Weeks": np.mean(auc_scores_weeks),
    "Objective": np.mean(auc_scores_objective),
    "Subjective": np.mean(auc_scores_subjective),
}

# Balkendiagramm für Mittelwerte der AUC-Scores
plt.figure(figsize=(10, 6))
bars = plt.bar(mean_auc_scores.keys(), mean_auc_scores.values(), color=["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"], alpha=0.85, edgecolor="black", linewidth=1.2)
plt.ylabel("Mean AUC Score", fontsize=12, fontweight="bold")
plt.xticks(fontsize=11)
plt.yticks(fontsize=11)
plt.title("Mittlere AUC Scores für verschiedene Feature-Sets", fontsize=14, fontweight="bold")
plt.ylim(min(mean_auc_scores.values()) - 0.01, max(mean_auc_scores.values()) + 0.01)
plt.grid(axis="y", linestyle="--", alpha=0.6)

# Werte über den Balken anzeigen
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.002, f"{yval:.4f}", ha="center", fontsize=11, fontweight="bold", color="black")

plt.show()


In [ ]:
import joblib

joblib.dump(xgb_models, 'xgb_models.pkl')
joblib.dump(xgb_models_days, 'xgb_models_days.pkl')
joblib.dump(xgb_models_weeks, 'xgb_models_weeks.pkl')
joblib.dump(xgb_models_objective, 'xgb_models_objective.pkl')
joblib.dump(xgb_models_subjective, 'xgb_models_subjective.pkl')

['xgb_models_subjective.pkl']